créditos:https://colab.research.google.com/drive/1FvBBw2JNXHy9WeJusI2PPr2PMnGVbbGM#scrollTo=GBotcBrgqR30 (lembrar de referenciar)

In [36]:
#%matplotlib inline
import pickle
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
#import seaborn as sns


In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
#carregando o dataset
data_set = '/content/drive/My Drive/WESAD/'
s2_path = data_set + 'S2.pkl'
#modificação necessária pois os dados estavam em py2 e o ambiente usa py3
with open(s2_path, 'rb') as file:
    s2_data = pickle.load(file, encoding='latin1')

In [39]:
#Separando dados de peito de dados de pulso (apenas dados de peito serão considerados conforme análise dos dados)
c_ax=s2_data['signal']['chest']['ACC'][0:,0]
c_ay=s2_data['signal']['chest']['ACC'][0:,1]
c_az=s2_data['signal']['chest']['ACC'][0:,2]
c_ecg=s2_data['signal']['chest']['ECG'][:,0]
c_emg=s2_data['signal']['chest']['EMG'][:,0]
c_eda=s2_data['signal']['chest']['EDA'][:,0]
c_temp=s2_data['signal']['chest']['Temp'][:,0]
c_resp=s2_data['signal']['chest']['Resp'][:,0]
label=s2_data['label']

# Alterar o objeto de dicionário para um array numpy e o array para um dataframe

*(referenciar)*
**Python | Numpy ndarray.T**
https://www.geeksforgeeks.org/python-numpy-ndarray-t/

**Gerando e Manipulando Arrays em Python**
https://towardsdatascience.com/numpy-array-cookbook-generating-and-manipulating-arrays-in-python-2195c3988b09

In [40]:
peito = [c_ax, c_ay, c_az, c_ecg, c_eda, c_emg, c_resp, c_temp, label]
p_array = np.array(peito)
p_array = p_array.T #(transposição matriz)
Colunas = ['c_ax', 'c_ay', 'c_az', 'c_ecg', 'c_eda', 'c_emg', 'c_resp', 'c_temp', 'label']

p_df = pd.DataFrame(p_array, columns = Colunas)
p_df

,c_ax,c_ay,c_az,c_ecg,c_eda,c_emg,c_resp,c_temp,label
0,0.9554,-0.2220,-0.5580,0.021423,5.250549,-0.004440,-1.148987,30.120758,0.0
1,0.9258,-0.2216,-0.5538,0.020325,5.267334,0.004349,-1.124573,30.129517,0.0
2,0.9082,-0.2196,-0.5392,0.016525,5.243301,0.005173,-1.152039,30.138214,0.0
3,0.8974,-0.2102,-0.5122,0.016708,5.249405,0.007187,-1.158142,30.129517,0.0
4,0.8882,-0.2036,-0.4824,0.011673,5.286407,-0.015152,-1.161194,30.130951,0.0
...,...,...,...,...,...,...,...,...,...
4255295,0.8750,-0.1234,-0.2974,-0.013138,0.400162,0.020370,-1.063538,31.457733,0.0
4255296,0.8750,-0.1262,-0.2988,-0.010345,0.355911,0.019592,-1.106262,31.476898,0.0
4255297,0.8718,-0.1238,-0.3042,-0.005447,0.360489,-0.017166,-1.103210,31.459229,0.0
4255298,0.8730,-0.1234,-0.3026,0.000137,0.365829,-0.028976,-1.086426,31.484283,0.0


In [41]:
p_df.describe().T

,count,mean,std,min,25%,50%,75%,max
c_ax,4255300.0,0.753866,0.175116,0.215000,0.621000,0.856800,0.892200,2.029800
c_ay,4255300.0,-0.133188,0.064173,-0.660000,-0.149800,-0.125600,-0.095800,0.539000
c_az,4255300.0,-0.422681,0.300236,-1.135400,-0.717200,-0.322200,-0.209800,1.246800
c_ecg,4255300.0,0.001212,0.154077,-1.499542,-0.067978,0.008240,0.044357,1.499313
c_eda,4255300.0,1.528387,1.238425,0.263214,0.799561,1.163101,1.546478,7.576752
c_emg,4255300.0,-0.002413,0.011652,-0.414963,-0.008331,-0.001602,0.003983,0.300980
c_resp,4255300.0,0.054906,2.917749,-27.903748,-1.481628,-0.157166,1.618958,27.378845
c_temp,4255300.0,30.876676,1.281106,28.045258,30.574951,31.289917,31.815125,34.370392
label,4255300.0,1.212370,1.606596,0.000000,0.000000,0.000000,2.000000,7.000000


In [42]:
p_df.columns

Index(['c_ax', 'c_ay', 'c_az', 'c_ecg', 'c_eda', 'c_emg', 'c_resp', 'c_temp',
       'label'],
      dtype='object')

Identificando os rótulos, usando apenas:
neutro: 0
stress: 1
diversão: 2

Os demais serão desconsiderados nesse estudo

In [43]:
neutro = p_df.loc[p_df['label'] == 0]
stress = p_df.loc[p_df['label'] == 1]
diversao = p_df.loc[p_df['label'] == 2]

p_loc = pd.concat([neutro, stress, diversao])
p_loc

,c_ax,c_ay,c_az,c_ecg,c_eda,c_emg,c_resp,c_temp,label
0,0.9554,-0.2220,-0.5580,0.021423,5.250549,-0.004440,-1.148987,30.120758,0.0
1,0.9258,-0.2216,-0.5538,0.020325,5.267334,0.004349,-1.124573,30.129517,0.0
2,0.9082,-0.2196,-0.5392,0.016525,5.243301,0.005173,-1.152039,30.138214,0.0
3,0.8974,-0.2102,-0.5122,0.016708,5.249405,0.007187,-1.158142,30.129517,0.0
4,0.8882,-0.2036,-0.4824,0.011673,5.286407,-0.015152,-1.161194,30.130951,0.0
...,...,...,...,...,...,...,...,...,...
2021977,0.8760,-0.1346,-0.3020,0.001328,3.075409,-0.004715,-1.161194,31.163483,2.0
2021978,0.8770,-0.1352,-0.2998,0.003433,3.059769,-0.003937,-1.124573,31.090057,2.0
2021979,0.8806,-0.1334,-0.2978,0.003616,3.078079,-0.009018,-1.097107,31.098877,2.0
2021980,0.8778,-0.1314,-0.2964,0.003525,3.058624,0.004944,-1.103210,31.101807,2.0


In [44]:
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

import keras
from keras.utils import to_categorical


# Normalizando dados: train test

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_axis.html (referenciar)

(referenciar)
https://scikit-learn.org/stable/modules/preprocessing.html 
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [45]:
x = p_loc.drop('label',axis=1).values
y = to_categorical(p_loc['label'])

scaler = MinMaxScaler(feature_range= (0,1))
x = scaler.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(x_train.shape)
print(y_test.shape)
evalSet = [(x_train, y_train), (x_test, y_test)]



(2699200, 8)
(674801, 3)


In [48]:
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

ValueError: ignored